# FASTEDDY TUTORIAL VISUALIZATION AND ANALYSIS NOTEBOOK

# 1. Import the tutorial helper-functions from python file

In [ ]:
import os, sys
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import math
from scipy.stats import skew
from scipy.stats import kurtosis
import matplotlib.colors as mcolors
import scipy.fftpack as fftpack
from scipy import interpolate
from fetutorialfunctions import *

In [ ]:
check_imports()

# 2. Define parameters

In [ ]:
##########################
# BEGIN USER EDIT REGION #
##########################
path_base = "INSERT_PATH_TO_YOUR_RUN_DIRECTORY"
path_end = "/output/"      # FOR EACH CASE, SUBDIRECTORY PATHS TO OUTPUT
save_plot_opt = 1          # 0-DO NOT SAVE PLOTS, 1-SAVE PLOTS

case = 'stable' # USER SELECTED CASE: 'stable', 'convective', or 'neutral'

if (case=='neutral'):
    zChoose = 100.0 # VERTICAL HEIGHT TO MAKE XY PLOTS
    z_max = 1000.0 # maximum height to plot in the vertical crossections/profiles
    cases_FE = ['Example01_NBL'] # SUBDIRECTORY PATHS TO EACH CASE
    output_base = ["FE_NBL."] # OUTPUT FILE NAMES FOR EACH CASE (NBL: NEUTRAL, CBL: CONVECTIVE, and SBL: STABLE)
    FE_timestep = ['630000'] # FINAL TIME STEPS OF CASES
    path_figure = path_base + cases_FE[0] + '/FIGS/' # BASE PATH TO FIGURES IF save_plot_opt=1
    FE_timestep_avg = ['540000','555000','570000','585000','600000','615000','630000'] # Output files used for temporal averaging
    sizeX_XZ = 22
    sizeY_XZ = 14
elif (case=='convective'):
    zChoose = 500.0 # VERTICAL HEIGHT TO MAKE XY PLOTS
    z_max = 3000.0 # maximum height to plot in the vertical crossections/profiles
    cases_FE = ['Example02_CBL'] # SUBDIRECTORY PATHS TO EACH CASE
    output_base = ["FE_CBL."] # OUTPUT FILE NAMES FOR EACH CASE (NBL: NEUTRAL, CBL: CONVECTIVE, and SBL: STABLE)
    FE_timestep = ['288000'] # FINAL TIME STEPS OF CASES
    path_figure = path_base + cases_FE[0] + '/FIGS/' # BASE PATH TO FIGURES IF save_plot_opt=1
    FE_timestep_avg = ['216000','228000','240000','252000','264000','276000','288000'] # Output files used for temporal averaging
    sizeX_XZ = 20
    sizeY_XZ = 20
elif (case=='stable'):
    zChoose = 50.0 # VERTICAL HEIGHT TO MAKE XY PLOTS
    z_max = 300.0 # maximum height to plot in the vertical crossections/profiles
    cases_FE = ['Example03_SBL'] # SUBDIRECTORY PATHS TO EACH CASE
    output_base = ["FE_SBL."] # OUTPUT FILE NAMES FOR EACH CASE (NBL: NEUTRAL, CBL: CONVECTIVE, and SBL: STABLE)
    FE_timestep = ['8640000'] # FINAL TIME STEPS OF CASES
    path_figure = path_base + cases_FE[0] + '/FIGS/' # BASE PATH TO FIGURES IF save_plot_opt=1
    FE_timestep_avg = ['7992000','8100000','8208000','8316000','8424000','8532000','8640000'] # Output files used for temporal averaging
    sizeX_XZ = 9
    sizeY_XZ = 20

########################
# END USER EDIT REGION #
########################

# 3. Open files

In [ ]:
n_cases = len(cases_FE)
files_FE = []
for cc in range(0,len(cases_FE)):
    files_FE.append(path_base+cases_FE[cc]+path_end+output_base[cc]+FE_timestep[cc])

print('files_FE=',files_FE)

path_figure = path_figure 
if not os.path.exists(path_figure):
    os.makedirs(path_figure)
    
case_tmp = 'case_open'
case_open = case_tmp + '=xr.open_dataset(files_FE[0])'
print(case_open)
exec(case_open)

# 4. Plot XY cross sections 

In [ ]:
plot_XY_UVWTHETA(case, case_open, zChoose, save_plot_opt, path_figure)

# 5. Plot XZ cross sections 

In [ ]:
plot_XZ_UVWTHETA(case, case_open, z_max, sizeX_XZ, sizeY_XZ, save_plot_opt, path_figure)

# 6. Plot mean vertical profiles of state variables

In [ ]:
FE_mean_tmp = compute_mean_profiles(case_open)

In [ ]:
numPlotsX = 1
numPlotsY = 3
sizeX = 12
sizeY = 6
fig,axs = plot_figureConfigure(numPlotsX,numPlotsY,sizeX,sizeY,styleFile='./feplot.mplstyle')
plot_mean_profiles(fig, axs, FE_mean_tmp, z_max, case, save_plot_opt, path_figure, 0)

# 7. Plot mean vertical profiles of turbulent quantities

In [ ]:
FE_turb_tmp=0.0
for dd in range(0,len(FE_timestep_avg)):
    file_FE_tmp = path_base+cases_FE[0]+path_end+output_base[0]+FE_timestep_avg[dd]
    print(file_FE_tmp)
    FE_turb_xr_tmp = xr.open_dataset(file_FE_tmp)
    FE_turb_tmp = compute_turb_profiles(FE_turb_xr_tmp, FE_turb_tmp)

In [ ]:
numPlotsX = 1
numPlotsY = 4
sizeX = 16
sizeY = 6
fig,axs = plot_figureConfigure(numPlotsX,numPlotsY,sizeX,sizeY,styleFile='./feplot.mplstyle')
plot_turb_profiles(fig, axs, case, FE_turb_tmp, z_max, save_plot_opt, path_figure)